# Chapter 17: Representation Learning dan Generative Learning
## Autoencoders & Generative Adversarial Networks (GANs)

Pada bab ini, kita membahas pendekatan **unsupervised learning** dan **generative modeling**
yang bertujuan untuk mempelajari representasi laten (*latent representations*) dari data.
Berbeda dengan supervised learning yang bergantung pada label, pendekatan ini berfokus
pada struktur internal data itu sendiri.

Dua keluarga model utama yang dibahas adalah **Autoencoders** dan
**Generative Adversarial Networks (GANs)**.
Autoencoder digunakan untuk kompresi, ekstraksi fitur, dan reduksi dimensi,
sedangkan GAN digunakan untuk menghasilkan data sintetis yang menyerupai data asli.

Pendekatan-pendekatan ini banyak digunakan dalam aplikasi modern seperti:
- Pengolahan dan rekonstruksi citra
- Deteksi anomali
- Data augmentation
- Pembuatan konten sintetis (gambar, audio, dan video)


### Fokus Pembelajaran

Pada akhir bab ini, pembaca diharapkan mampu:
1. Memahami prinsip kerja **Autoencoder** dan efek *bottleneck*.
2. Menjelaskan perbedaan **Undercomplete**, **Stacked**, dan **Convolutional Autoencoder**.
3. Memahami peran regularisasi melalui **Denoising** dan **Sparse Autoencoder**.
4. Menjelaskan konsep probabilistik pada **Variational Autoencoder (VAE)**.
5. Memahami dinamika pelatihan **Generative Adversarial Networks (GAN)**.
6. Mengenal **DCGAN** sebagai standar GAN berbasis konvolusi untuk citra.


## 1. Undercomplete Autoencoders

Undercomplete Autoencoder merupakan bentuk autoencoder paling dasar,
di mana dimensi *latent space* (bottleneck) dibuat lebih kecil
dibandingkan dengan dimensi input.

Dengan keterbatasan ini, jaringan dipaksa untuk mempelajari
representasi data yang paling informatif,
bukan sekadar menyalin input ke output.
Jika seluruh lapisan bersifat linear dan fungsi loss adalah MSE,
model ini secara teoritis setara dengan **Principal Component Analysis (PCA)**.

Pendekatan ini sering digunakan sebagai langkah awal
untuk memahami struktur data berdimensi tinggi.


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Membuat data sintetis 3D yang terletak pada manifold 2D
def generate_3d_data(m, w1=0.1, w2=0.3, noise=0.1):
    angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
    data = np.empty((m, 3))
    data[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
    data[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
    data[:, 2] = data[:, 0] * w1 + data[:, 1] * w2 + noise * np.random.randn(m)
    return data

X_train = generate_3d_data(100)
X_train = X_train - X_train.mean(axis=0)

# Autoencoder linear dengan bottleneck 2 neuron
encoder = keras.models.Sequential([
    keras.layers.Dense(2, input_shape=[3])
])

decoder = keras.models.Sequential([
    keras.layers.Dense(3, input_shape=[2])
])

autoencoder = keras.models.Sequential([encoder, decoder])

autoencoder.compile(
    loss="mse",
    optimizer=keras.optimizers.SGD(learning_rate=0.1)
)

history = autoencoder.fit(X_train, X_train, epochs=20, verbose=0)

print("Pelatihan Autoencoder selesai. Loss akhir:", history.history["loss"][-1])


## 2. Stacked Autoencoders

Stacked Autoencoder terdiri dari beberapa lapisan tersembunyi
yang memungkinkan model mempelajari representasi yang lebih kompleks dan hierarkis.
Setiap lapisan mengekstraksi fitur dari representasi lapisan sebelumnya.

Keuntungan utama pendekatan ini adalah fleksibilitas dan daya representasi yang tinggi.
Namun, jika kapasitas jaringan terlalu besar,
autoencoder dapat mengalami **overfitting**
dan hanya menghafal data pelatihan tanpa benar-benar belajar pola yang bermakna.

Oleh karena itu, pemilihan arsitektur dan regularisasi menjadi sangat penting.


In [ ]:
# Memuat dataset Fashion MNIST
(X_train_full, _), (X_test, _) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255

X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]

def build_stacked_autoencoder():
    encoder = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(100, activation="selu"),
        keras.layers.Dense(30, activation="selu"),
    ])

    decoder = keras.models.Sequential([
        keras.layers.Dense(100, activation="selu", input_shape=[30]),
        keras.layers.Dense(28 * 28, activation="sigmoid"),
        keras.layers.Reshape([28, 28])
    ])

    return keras.models.Sequential([encoder, decoder])

stacked_ae = build_stacked_autoencoder()
stacked_ae.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=1.5)
)

print("Stacked Autoencoder berhasil dibangun.")


## 3. Convolutional Autoencoders

Convolutional Autoencoder dirancang khusus untuk data citra.
Alih-alih menggunakan lapisan Dense,
model ini memanfaatkan **Conv2D** pada encoder
dan **Conv2DTranspose** pada decoder.

Pendekatan ini mempertahankan struktur spasial gambar
dan umumnya menghasilkan rekonstruksi yang jauh lebih baik
dibanding autoencoder berbasis fully-connected.


In [ ]:
conv_encoder = keras.models.Sequential([
    keras.layers.Reshape([28, 28, 1], input_shape=[28, 28]),
    keras.layers.Conv2D(16, 3, padding="same", activation="selu"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(32, 3, padding="same", activation="selu"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(64, 3, padding="same", activation="selu"),
    keras.layers.MaxPool2D(2)
])

conv_decoder = keras.models.Sequential([
    keras.layers.Conv2DTranspose(32, 3, strides=2, padding="valid", activation="selu", input_shape=[3, 3, 64]),
    keras.layers.Conv2DTranspose(16, 3, strides=2, padding="same", activation="selu"),
    keras.layers.Conv2DTranspose(1, 3, strides=2, padding="same", activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])
conv_ae.summary()


## 4. Variational Autoencoders (VAE)

Berbeda dengan autoencoder klasik,
Variational Autoencoder (VAE) mempelajari
distribusi probabilitas pada latent space,
bukan sekadar satu titik deterministik.

Pendekatan ini memungkinkan kita
melakukan *sampling* untuk menghasilkan data baru.
VAE menjadi fondasi penting bagi banyak model generatif modern,
meskipun hasil rekonstruksinya cenderung lebih halus (blur).


## 5. Generative Adversarial Networks (GANs)

GAN merupakan pendekatan generatif berbasis permainan dua pemain (*two-player game*).
Model ini terdiri dari:
- **Generator**, yang berusaha menciptakan data palsu menyerupai data asli.
- **Discriminator**, yang bertugas membedakan data asli dan data palsu.

Kedua jaringan dilatih secara bersamaan dalam skema kompetitif,
di mana peningkatan performa satu model
akan memaksa model lainnya untuk ikut membaik.


In [ ]:
codings_size = 30

generator = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid")
])

discriminator.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop"
)

discriminator.trainable = False

gan = keras.models.Sequential([generator, discriminator])
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

print("Model GAN dasar berhasil didefinisikan.")


## 6. Tantangan Melatih GAN

Meskipun sangat kuat, GAN terkenal sulit untuk dilatih.
Beberapa tantangan utama meliputi:
1. **Mode Collapse**: Generator hanya menghasilkan satu jenis output.
2. **Ketidakseimbangan Pelatihan**: Discriminator terlalu kuat atau terlalu lemah.
3. **Instabilitas Loss**: Loss tidak selalu mencerminkan kualitas hasil.

Berbagai teknik seperti DCGAN, WGAN, dan spectral normalization
dikembangkan untuk mengatasi masalah ini.


## Kesimpulan Praktis

- Autoencoder sangat berguna untuk reduksi dimensi dan ekstraksi fitur.
- VAE memungkinkan eksplorasi latent space secara probabilistik.
- GAN mampu menghasilkan data sintetis yang sangat realistis.
- Namun, GAN membutuhkan strategi pelatihan yang hati-hati.
